https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_adaptive_rag_local/  

https://www.youtube.com/watch?v=bq1Plo2RhYI  


In [23]:
from langchain_ollama import ChatOllama

llm_model = "llama3.2:3b-instruct-fp16"

llm_json_mode = ChatOllama(model=llm_model, temperature=0, format='json')

In [24]:
from langchain_nomic.embeddings import NomicEmbeddings

embeddings = NomicEmbeddings(model='nomic-embed-text-v1.5', inference_mode='local')

In [25]:
from langchain.schema import Document
import json

def prepare_qa_documents(file_path):
    with open(file_path, 'r') as f:
        qa_data = json.load(f)
    
    documents = [
        Document(
            page_content= f"Q: {item['question']} A: {item['answer']}",
            metadata={"question": item["question"]}
        )
        for item in qa_data
    ]
    
    return documents

documents = prepare_qa_documents("../data/home0001qa.json")
print(documents[:1])

[Document(metadata={'question': 'Do i own my 0001 home outright?'}, page_content='Q: Do i own my 0001 home outright? A: When you buy a 0001 home, you own the title in the traditional way. If you need, we’ll help you find the right mortgage and can recommend real estate lawyers. You keep full legal ownership of your home, with the added benefit that you can spend time in other locations whenever you want.')]


In [26]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents, embeddings)

retriever = vectorstore.as_retriever(k=3)

In [27]:
retriever.invoke("Do i own my 0001 home outright?")

[Document(metadata={'question': 'Do i own my 0001 home outright?'}, page_content='Q: Do i own my 0001 home outright? A: When you buy a 0001 home, you own the title in the traditional way. If you need, we’ll help you find the right mortgage and can recommend real estate lawyers. You keep full legal ownership of your home, with the added benefit that you can spend time in other locations whenever you want.'),
 Document(metadata={'question': 'Do i fully own my 0001 home?'}, page_content="Q: Do i fully own my 0001 home? A: When you buy a 0001 home you own it the normal way. You can do whatever you want with it. Buy it. Sell it. Renovate it. It's not a timeshare or some bs rental scheme. You own the title to the home. It's yours."),
 Document(metadata={'question': 'Do i have to make my 0001 home available, and can i rent out or sell my 0001 home?'}, page_content="Q: Do i have to make my 0001 home available, and can i rent out or sell my 0001 home? A: You own your home. You're in control. If

In [28]:
# Doc grader instructions
doc_grader_instructions = """You are a grader assessing relevance of a retrieved document to a user question.

If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant."""

# Grader prompt
doc_grader_prompt = """Here is the retrieved document: \n\n {document} \n\n Here is the user question: \n\n {question}. 

This carefully and objectively assess whether the document contains at least some information that is relevant to the question.

Return JSON with single key, binary_score, that is 'yes' or 'no' score to indicate whether the document contains at least some information that is relevant to the question."""

In [29]:
from langchain_core.messages import HumanMessage, SystemMessage

# Test
question = "Does furniture come included?"
docs = retriever.invoke(question)
doc_txt = docs[0].page_content
doc_grader_prompt_formatted = doc_grader_prompt.format(
    document=doc_txt, question=question
)
result = llm_json_mode.invoke(
    [SystemMessage(content=doc_grader_instructions)]
    + [HumanMessage(content=doc_grader_prompt_formatted)]
)

print(doc_txt)
print(json.loads(result.content))

Q: Does furniture come included? A: Yes, each new home comes fully furnished and equipped so that you can move in easily with just your suitcase. Each interior is designed for flexibility and functionality and we work directly with designers from our community to source pieces often straight from their studios.
{'binary_score': 'yes'}


In [34]:
def grade_documents(documents, question):
  
    # Score each doc
    filtered_docs = []

    for d in documents:
        doc_grader_prompt_formatted = doc_grader_prompt.format(
            document=d.page_content, question=question
        )
        result = llm_json_mode.invoke(
            [SystemMessage(content=doc_grader_instructions)]
            + [HumanMessage(content=doc_grader_prompt_formatted)]
        )
        grade = json.loads(result.content)["binary_score"]
        # Document relevant
        if grade.lower() == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        # Document not relevant
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            # We do not include the document in filtered_docs
            continue

    return {"documents": filtered_docs}

In [36]:
question = "Do i own my 0001 home outright?"
retrieved_docs = retriever.invoke(question)
filtered_docs = grade_documents(retrieved_docs, question)


---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---


In [31]:
test_string = """ 
    this is a {test}
"""

formatted_test_string = test_string.format(test="hello")

print(formatted_test_string)

 
    this is a hello



In [32]:
# TO DO: HALLUCINATION GRADER
# ANSWER GRADER 